In [1]:
import os
import xmlrpc.client
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


api_url = os.environ.get('ODOO_URL_API')
api_db = os.environ.get('ODOO_DB_API')
# api_db = os.environ.get('ODOO_DB_PRUEBA_API')
api_username = os.environ.get('ODOO_USERNAME_API')
api_clave = os.environ.get('ODOO_CLAVE_API')


common = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/common')
uid = common.authenticate(api_db, api_username, api_clave, {})
models = xmlrpc.client.ServerProxy(f'{api_url}/xmlrpc/2/object')

In [2]:
def if_list_gt0_idex (item: dict, key: str, index:int ) -> None | int:
        val = item[key]

        if val:
            if len(val) == 0:
                return None
            else:
                return val[index]
        else:
            return None

In [12]:
def search_fact_func(param_mes: int, param_dias: None | list[int] = None) -> list[str]:
    
    if type(param_mes) != int or param_mes < 1 or param_mes > 12:
        raise Exception (f'El mes es incorrecto. El párametro "mes" debe ser un número entero entre 1 y 12. Escribiste: {param_mes}')
    

    if param_dias is None:
        param_dia_ini = datetime(2024, param_mes, 1)
        param_dia_fin = datetime(2024, param_mes + 1, 1) - timedelta(days= 1)
    
    elif len(param_dias) != 2:
        raise Exception (f'El párametro "días_del_mes" debe ser una lista de 2 elementos. El día inicial de búsqueda se escribe en el índice 0 de la lista, el día final de búsqueda en el índice 1. La lista tiene: {len(param_dias)} de elementos')
    
    elif type(param_dias[0]) != int or type(param_dias[1]) != int:
        raise Exception (f'El párametro "día_inicial_de_búsqueda" y "día_final_de_búsqueda" sólo pueden ser números enteros.')

    elif param_dias[0] > param_dias[1]:
        raise Exception (f'El párametro "día_inicial_de_búsqueda" no debe ser mayor al parámetro "día_final_de_búsqueda". El día inicial de búsqueda se escribe en el índice 0 de la lista, el día final de búsqueda en el índice 1.')
    
    else:
        try:
            param_dia_ini = datetime(2024, param_mes, param_dias[0])
        except:
            raise Exception (f'El párametro "día_inicial_de_búsqueda" es incorrecto. La fecha "día: {param_dias[0]}, mes: {param_mes}, año 2024" no existe')
        try:
            param_dia_fin = datetime(2024, param_mes, param_dias[1])
        except:
            raise Exception (f'El párametro "día_final_de_búsqueda" es incorrecto. La fecha "día: {param_dias[1]}, mes: {param_mes}, año 2024" no existe')

    
    search_fact = ["&", "&", "&",
                ("state", "=", "posted"),
                ("invoice_date", ">=", param_dia_ini.strftime('%Y-%m-%d')), 
                ("invoice_date", "<=", param_dia_fin.strftime('%Y-%m-%d')), 
                ("journal_id", "in", [10, 90, 30, 97])]
    
    
    
    return search_fact

In [47]:
def fact_doc_df_func(param_mes: int, param_dias: None | list[int] = None) -> list[pd.DataFrame, list[int], list[int]]:
    

        search_fact = search_fact_func(param_mes, param_dias = None)


        fact_doc_fields = [
                'name',
                'invoice_date',
                'state',
                'reversed_entry_id',
                'reversal_move_id',
                'journal_id',
                'company_id',
                'invoice_origin',
                'pos_order_ids',
                'line_ids',
                'partner_id',
                'move_type',
                'invoice_user_id'
                ]

        fact_doc_ids1 = models.execute_kw(api_db, uid, api_clave, 'account.move', 'search', [search_fact])
        fact_doc_ids2 = models.execute_kw(api_db, uid, api_clave, 'account.move', 'search', [[("reversal_move_id", "=", fact_doc_ids1)]])
        fact_doc_ids = list(set(fact_doc_ids1) | set(fact_doc_ids2))

        fact_doc_json = models.execute_kw(api_db, uid, api_clave, 'account.move', 'read', [fact_doc_ids], {'fields': fact_doc_fields})

        data_fact = []
        fact_line_ids = []
        pos_doc_ids = []

        for fact in fact_doc_json:
                if fact['pos_order_ids']:
                        pos_doc_ids.append(fact['pos_order_ids'][0])

                for line in fact['line_ids']:
                        new = {}
                        new['fact_doc_id'] = fact['id']
                        new['name'] = fact['name']
                        new['invoice_date'] = fact['invoice_date']
                        new['state'] = fact['state']
                        new['invoice_origin'] = fact['invoice_origin']
                        new['module_origin'] = None
                        new['pos_doc_id'] = if_list_gt0_idex(fact, 'pos_order_ids', 0)
                        new['move_type'] = fact['move_type']
                        new['reversal_move_id'] = if_list_gt0_idex(fact, 'reversal_move_id', 0)
                        new['reversed_entry_id'] = if_list_gt0_idex(fact, 'reversed_entry_id', 0)
                        new['journal_id'] = fact['journal_id'][0]
                        new['company_id'] = fact['company_id'][0]
                        new['partner_id'] = fact['partner_id'][0]
                        new['fact_line_id'] = line
                        new['invoice_user_id'] = fact['invoice_user_id'][0]

                        if not fact['invoice_origin']:
                                new['module_origin'] = 'Contabilidad'
                        elif fact['invoice_origin'][:2] in ['Pd', 'Sh']:
                                new['module_origin'] = 'PdV'
                        elif fact['invoice_origin'][0] == 'S':
                                new['module_origin'] = 'Ventas'


                        fact_line_ids.append(line)
                        data_fact.append(new)


        fact_doc_df = pd.DataFrame(data_fact)
        
        
        for col in fact_doc_df.columns:
                if fact_doc_df[col].dtype == 'int64' or fact_doc_df[col].dtype == 'float64':
                        fact_doc_df[col] = fact_doc_df[col].astype('Int64')
        
        fact_doc_df['invoice_date'] = pd.to_datetime(fact_doc_df['invoice_date'], format='%Y-%m-%d')
        fact_doc_df.loc[fact_doc_df['invoice_origin'] == False , ['invoice_origin',]] = pd.NA
        

        return fact_doc_df, fact_line_ids, pos_doc_ids

In [ ]:
fact_doc_df, fact_line_ids, pos_doc_ids = fact_doc_df_func(2)
pos_doc_ids

In [9]:
fact_line_fields = [
    'product_id',
    'quantity',
    'price_unit',
    'discount',
    'account_id',
    'price_subtotal',
    'sale_line_ids'
]

fact_line_json = models.execute_kw(api_db, uid, api_clave, 'account.move.line', 'read', [fact_line_ids], {'fields': fact_line_fields})

In [10]:
data_line_fact = []

for fact_line in fact_line_json:
    if fact_line['account_id'] and fact_line['account_id'][0] in [85, 197]:
        new = {}
        new['fact_line_id'] = fact_line['id']
        new['product_id'] = fact_line['product_id'][0]
        new['quantity'] = fact_line['quantity']
        new['price_unit'] = fact_line['price_unit']
        new['discount'] = fact_line['discount'] / 100
        new['price_subtotal'] = fact_line['price_subtotal']
        new['sale_line_id_fact'] = if_list_gt0_idex(fact_line, 'sale_line_ids', 0)

        data_line_fact.append(new)


fact_line_df = pd.DataFrame(data_line_fact)


fact_line_df['fact_line_id'] = fact_line_df['fact_line_id'].astype('Int64')
fact_line_df.loc[fact_line_df['product_id'] == False, ['product_id',]] = pd.NA
fact_line_df['product_id'] = fact_line_df['product_id'].astype('Int64')
fact_line_df['sale_line_id_fact'] = fact_line_df['sale_line_id_fact'].astype('Int64')

In [11]:
fact_df = fact_doc_df.merge(fact_line_df, how='right', on='fact_line_id')

In [12]:
pos_doc_ids = []

for fact in fact_doc_json:
    if fact['pos_order_ids']:
        pos_doc_ids.append(fact['pos_order_ids'][0])

In [13]:
pos_doc_name_extra = list(fact_df.loc[(fact_df['module_origin'] == 'PdV') & (fact_df['pos_doc_id'].isna()), 'invoice_origin'].unique())

search_pos = [
    ("name", "in", pos_doc_name_extra),
]

pos_doc_fields = [
    'name'
]

pos_doc_ids_extra = models.execute_kw(api_db, uid, api_clave, 'pos.order', 'search', [search_pos])
pos_doc_json_extra = models.execute_kw(api_db, uid, api_clave, 'pos.order', 'read', [pos_doc_ids_extra], {'fields': pos_doc_fields})

for pos in pos_doc_json_extra:
    pos_doc_ids.append(pos['id'])

In [14]:
for item in pos_doc_json_extra:
    fact_df.loc[fact_df['invoice_origin'] == item['name'], 'pos_doc_id'] = item['id']

In [15]:
pos_line_fields = [
    'order_id',
    'sale_order_line_id',
    'refund_orderline_ids',
    'refunded_orderline_id',
]


pos_line_ids = models.execute_kw(api_db, uid, api_clave, 'pos.order.line', 'search', [[("order_id.id", "=", pos_doc_ids)]])
pos_line_json = models.execute_kw(api_db, uid, api_clave, 'pos.order.line', 'read', [pos_line_ids], {'fields': pos_line_fields})

In [16]:
data_pos_line = []

for pos in pos_line_json:
    new = {}
    new['pos_line_id'] = pos['id']
    new['pos_doc_id'] = pos['order_id'][0]
    new['sale_line_id_pos'] = if_list_gt0_idex(pos, 'sale_order_line_id', 0)
    new['refund_orderline_ids'] = if_list_gt0_idex(pos, 'refund_orderline_ids', 0)
    new['refunded_orderline_id'] = if_list_gt0_idex(pos, 'refunded_orderline_id', 0)
    

    data_pos_line.append(new)

dfpos_line = pd.DataFrame(data_pos_line)
dfpos_line['sale_line_id_pos'] = dfpos_line['sale_line_id_pos'].astype('Int64')
dfpos_line['pos_line_id'] = dfpos_line['pos_line_id'].astype('Int64')
dfpos_line['refund_orderline_ids'] = dfpos_line['refund_orderline_ids'].astype('Int64')
dfpos_line['refunded_orderline_id'] = dfpos_line['refunded_orderline_id'].astype('Int64')

In [17]:
fact_pos_doc_df = fact_df[(~fact_df['pos_doc_id'].isna()) & (fact_df['move_type'] == 'out_invoice')][['fact_doc_id','name', 'fact_line_id', 'pos_doc_id', 'product_id']]

In [18]:
group_fact_pos_df = fact_pos_doc_df.groupby('pos_doc_id').count()['fact_doc_id']
group_pos_line_df = dfpos_line.groupby('pos_doc_id').count()['pos_line_id']

check_total_size = len(group_fact_pos_df) == len(group_pos_line_df)

groups_concat = pd.concat([group_fact_pos_df, group_pos_line_df], axis=1)
groups_concat['lines_per_doc'] = groups_concat['fact_doc_id'] == groups_concat['pos_line_id']
check_each_document_size = len(groups_concat[~groups_concat['lines_per_doc']]) == 0

Para poder hacer un <font color="#14E4FA">merge</font> entre `fact_df`, `pos_line_df` se busca hacer un id temporal `id_vinculo` que sirva de vínculo entre ambos dataframes.

Para generar `id_vinculo`, se utiliza el único campo en común para ambos dataframes <font color="#14FA4C">"pos_order_ids"</font> en `fact_df` y <font color="#14FA4C">"pos_doc_id"</font> en `pos_line_df`.

Se generó la función `id_vinculo_generator`, la cual recibe tres parámetros: el dataframe, el nombre de la columna donde se almacenan los ids de los documentos del pos y el nombre de la columna donde se almacenan los ids de las líneas de su modelo.

In [20]:
def id_vinculo_generator(df_base: pd.DataFrame, pos_doc_column_name: str, df_id_line_name:str ) -> pd.DataFrame:
    
    df = df_base.sort_values(by=[pos_doc_column_name, df_id_line_name])
    df['id_relative_pos'] = None

    pos_orders = df[pos_doc_column_name].unique()

    for pos in pos_orders:
        mini_df = df.loc[df[pos_doc_column_name] == pos]
        df.loc[df[pos_doc_column_name] == pos, 'id_relative_pos'] = [i for i in range(len(mini_df))]


    df['id_vinculo'] = df[pos_doc_column_name].astype(str) + '-' + df['id_relative_pos'].astype(str)

    return df

Se procede a ejecutar la función anterior para cada dataframe. Se guardan sus resultados respectivos en `fact_pos_link_df` y `pos_line_link_df`

In [21]:
fact_pos_doc_link_df = id_vinculo_generator( fact_pos_doc_df, 'pos_doc_id', 'fact_line_id')
pos_line_link_df = id_vinculo_generator(dfpos_line, 'pos_doc_id', 'pos_line_id')

Se procede a generar el merge de ambos dataframes de linea, dando como resultado `fac_pos_linked_df`.

In [22]:
fac_pos_linked_df =  fact_pos_doc_link_df.merge(pos_line_link_df, how='outer', on='id_vinculo')

Se procede a integrar `fac_pos_linked_df` al dataframe general `fact_df` por medio de un <font color="#14E4FA">merge</font> tipo "left join" en su campo de línea de factura <font color="#14FA4C">"id_x"</font>, obteniendo `fact_pos_df`. Al haber vinculado <font color="#F414FA">pos.order.line</font> al dataframe general, cada línea de venta (ya sea que venga del módulo de "Ventas" o del módulo de "PdV") tiene ya un id de línea del modelo <font color="#F414FA">sale.order.line</font>, que es donde se encuentra el nombre de la vendedora.

In [23]:
cols_to_keep = ['fact_line_id', 'pos_line_id', 'sale_line_id_pos', 'refund_orderline_ids', 'refunded_orderline_id']
fact_pos_df = fact_df.merge(fac_pos_linked_df[cols_to_keep], how='left', on='fact_line_id')

Al haber hecho merge en dos dataframes diferentes en `fact_pos_df`, se tienen los ids del modelo <font color="#F414FA">sale.order.line</font> en diferentes columnas. Se procede a juntar dichos ids en una sóla columna <font color="#14FA4C">"sale_line_join_id"</font>.

Se genera `ids_sale_line` que es una lista de los ids únicos para poder utilizarlos a posterior.

In [24]:
fact_pos_df['sale_line_id'] = None
fact_pos_df.loc[(~fact_pos_df['sale_line_id_fact'].isna() & fact_pos_df['sale_line_id_pos'].isna()), 'sale_line_id'] = fact_pos_df['sale_line_id_fact']
fact_pos_df.loc[(fact_pos_df['sale_line_id_fact'].isna() & ~fact_pos_df['sale_line_id_pos'].isna()), 'sale_line_id'] = fact_pos_df['sale_line_id_pos']
fact_pos_df['sale_line_id'] = fact_pos_df['sale_line_id'].astype('Int64')

fact_pos_df.drop(columns=['sale_line_id_fact', 'sale_line_id_pos'], inplace=True)

sale_line_ids = []
for sid in fact_pos_df.loc[~fact_pos_df['sale_line_id'].isna(), 'sale_line_id'].unique():
    sale_line_ids.append(int(sid))

Se especifican los campos necesarios `sale_line_fields` para que con `sale_line_ids` (que se generó arriba) se pueda obtener la información del modelo <font color="#F414FA">sale.order.line</font>. Esto devuelve una lista de diccionarios con la información de cada línea de venta, que se guarda en `sale_line_json`.

In [25]:
sale_line_fields = [
    'salesman_id',
]

sale_line_json = models.execute_kw(api_db, uid, api_clave, 'sale.order.line', 'read', [sale_line_ids], {'fields': sale_line_fields})

Se procede a generar el DataFrame `sale_line_df`. Para esto se preparan los datos con un ciclo <font color="#14E4FA">for</font> donde se manipula `sale_line_json`.

In [26]:
data_sale_line = []

for sale in sale_line_json:
    new = {}
    new['sale_line_id'] = sale['id']
    new['salesman_id'] = sale['salesman_id'][0]

    data_sale_line.append(new)


sale_line_df = pd.DataFrame(data_sale_line)
sale_line_df['salesman_id'] = sale_line_df['salesman_id'].astype('Int64')

Se procede a integrar `sale_line_df` al dataframe general `fact_pos_df` por medio de un <font color="#14E4FA">merge</font> tipo "left join" en su campo de línea de sale <font color="#14FA4C">"sale_line_id"</font>, obteniendo `complete_df`.

In [27]:
complete_df = fact_pos_df.merge(sale_line_df, how='left', on='sale_line_id').set_index('name')

Se filtra las líneas de factura que son de reversiones o notas de crédito/devolución. Estas líneas tienen que cambiar su signo a negativo para que la analítica de sumas y acumulados refleje la realidad en `complete_df`.

In [28]:
complete_df.loc[complete_df['move_type'] == 'out_refund', ['quantity', 'price_subtotal']] = complete_df.loc[complete_df['move_type'] == 'out_refund', ['quantity', 'price_subtotal']] * -1

Se ocupa corregir todas las líneas que en el campo 'salesman_id' no tienen información en `complete_df`. 

Se empieza con las líneas donde el campo 'module_origin' vengan de contabilidad o ventas. Debido a que en estas líneas el campo 'invoice_user_id' sí refleja a la vendedora de manera correcta, sólo se copia  'invoice_user_id' al campo 'salesman_id'.

In [29]:
complete_df.loc[complete_df['module_origin'] != 'PdV', 'salesman_id'] = complete_df.loc[complete_df['module_origin'] != 'PdV', 'invoice_user_id']

Al corregir líneas que vienen de Contabilidad o Ventas, sólo faltan las que vienen de PdV. Por lo tanto ahora se trabajan las líneas que no tienen 'salesman_id, que son movimientos tipo "out-invoice" y que sí tienen 'pos_line_id' para ser buscadas en `pos_line_json`.

Debido a que son facturas, estás líneas sin 'salesman_id' son adiciones que la cajera hizo al documento pos. Se trata de encontrar una línea hermana en este mismo documento, así que se trae del json todas las líneas que coinciden con el campo 'pos_doc_id'. Se busca una línea hermana que no tenga que tenga información en el campo 'sale_line_id' y se busca esta línea de venta en `complete_df`. Se copia el campo 'salesman_id' de esta línea hermana a la línea original. Para evitar que pase esto mismo con cada línea que pueda tener el documento pos, se genera un "break".

In [30]:
pos_doc_id_not_saleline, pos_line_id_not_saleline = complete_df.loc[(complete_df['salesman_id'].isna()) &(complete_df['module_origin'] == 'PdV') & (complete_df['move_type'] == 'out_invoice') & (~complete_df['pos_line_id'].isna()), ['pos_doc_id', 'pos_line_id']].items()

for i in range(len(pos_line_id_not_saleline[1])):
    for pos in pos_line_json:
        if pos['order_id'][0] == pos_doc_id_not_saleline[1].iloc[i]:
            if pos['sale_order_line_id']:
                complete_df.loc[complete_df['pos_line_id'] == pos_line_id_not_saleline[1].iloc[i], 'salesman_id'] = complete_df.loc[complete_df['sale_line_id'] == pos['sale_order_line_id'][0], 'salesman_id'].iloc[0]
                break

Ahora se corrigen todas las líneas que vienen de PdV, no tienen 'salesman_id', que son movimientos tipo "out-refund" y que sí tienen 'pos_line_id' para ser buscadas en `pos_line_json`.

Debido a que esta línea de devoluciones no siempre viene de facturas de este mes o puede que vengan de faturas cancelada, `pos_line_json` no tiene información sobre sus líneas 'refunded' o devueltas. Por lo tanto, se genera un ciclo "for" para buscar el campo 'refunded_orderline_id' en cada "id" de `pos_line_json`:
    
-> Si existe la línea 'refunded' en 'pos_line_id':
    Se copia el campo 'salesman_id' de la línea original a la línea de devolución.

-> Si no existe la línea 'refunded' en 'pos_line_id':
    Se genera una llamada a la API de Odoo por el pos_line_id original, si este devuelve que hay información en su campo 'sale_order_line_id', se genera una segunda llamada a la API de Odoo por el sale_line_id y traemos el campo 'salesman_id'. Este campo 'salesman_id' de la línea original se copia la línea de devolución.

In [31]:
pos_line_id_not_saleline, refunded_id_not_saleline = complete_df.loc[(complete_df['salesman_id'].isna()) &(complete_df['module_origin'] == 'PdV') & (complete_df['move_type'] == 'out_refund') & (~complete_df['pos_line_id'].isna()), ['pos_line_id', 'refunded_orderline_id']].items()

for i in range(len(pos_line_id_not_saleline[1])):
    if not complete_df.loc[complete_df['pos_line_id'] == refunded_id_not_saleline[1].iloc[i]].empty:
        complete_df.loc[complete_df['pos_line_id'] == pos_line_id_not_saleline[1].iloc[i], 'salesman_id'] = complete_df.loc[complete_df['pos_line_id'] == refunded_id_not_saleline[1].iloc[i], 'salesman_id'].iloc[0]
    else:
        pos_line_json_not_found = models.execute_kw(api_db, uid, api_clave, 'pos.order.line', 'read', [[int(refunded_id_not_saleline[1].iloc[i])]])
        res_id = pos_line_json_not_found[0]['sale_order_line_id']
        if res_id:
            sale_line_json_pos_not_found = models.execute_kw(api_db, uid, api_clave, 'sale.order.line', 'read', [[ res_id[0]]], {'fields': sale_line_fields})
            complete_df.loc[complete_df['pos_line_id'] == pos_line_id_not_saleline[1].iloc[i], 'salesman_id'] = sale_line_json_pos_not_found[0]['salesman_id'][0]

Por último para las líneas que vienen de PdV, no tienen 'salesman_id' y son movimientos tipo "out-refund" sólo faltan las que no tienen 'pos_line_id'. Al no existir información en este campo, no se puede buscar la línea en `pos_line_json`.

Con estas líneas se busca en `compelte_df` el documento pos del cuál se hizo la devolución (pues vienen de PdV). Se encuentra el dataframe que lleva este documento, se quita todo lo que es devolución y con la información que queda (las ventas) se trata de encontrar el mismo id de producto (pues carecemos de una línea en la cuál empatar venta-devolución). Si se encuentra esta línea, se toma el campo 'salesman_id' de la línea original y se copia a la línea de devolución.

Las líneas que no se encuetran en `compelte_df` se trabajan en el siguiente snipet.

In [32]:
fact_line_id_not_saleline, pos_doc_id_not_saleline, product_id_not_saleline = complete_df.loc[(complete_df['salesman_id'].isna()) & (complete_df['pos_line_id'].isna()), ['fact_line_id','pos_doc_id', 'product_id']].items()

for i in range(len(fact_line_id_not_saleline[1])):
    if not complete_df.loc[(complete_df['pos_doc_id'] == pos_doc_id_not_saleline[1].iloc[i] ) & (complete_df['product_id'] == product_id_not_saleline[1].iloc[i]) & (complete_df['move_type'] != 'out_refund')].empty:
        complete_df.loc[complete_df['fact_line_id'] == fact_line_id_not_saleline[1].iloc[i], 'salesman_id'] = complete_df.loc[(complete_df['pos_doc_id'] == pos_doc_id_not_saleline[1].iloc[i] ) & (complete_df['product_id'] == product_id_not_saleline[1].iloc[i]) & (complete_df['move_type'] != 'out_refund'), 'salesman_id'].iloc[0]

In [33]:
complete_df.loc[complete_df['salesman_id'].isna()]

,fact_doc_id,invoice_date,state,invoice_origin,module_origin,pos_doc_id,move_type,reversal_move_id,reversed_entry_id,journal_id,...,product_id,quantity,price_unit,discount,price_subtotal,pos_line_id,refund_orderline_ids,refunded_orderline_id,sale_line_id,salesman_id
name,,,,,,,,,,,,,,,,,,,,,
RF2-CC/2024/00034,28937,2024-02-21,posted,PdV SJC/3754 REEMBOLSO,PdV,6549,out_refund,<NA>,28811,90,...,8506,-300.00,15.39,0.2,-3693.60,16307,<NA>,16199,<NA>,<NA>
RF2-CC/2024/00034,28937,2024-02-21,posted,PdV SJC/3754 REEMBOLSO,PdV,6549,out_refund,<NA>,28811,90,...,8092,-300.00,7.21,0.2,-1730.40,16308,<NA>,16200,<NA>,<NA>
RF2-CC/2024/00034,28937,2024-02-21,posted,PdV SJC/3754 REEMBOLSO,PdV,6549,out_refund,<NA>,28811,90,...,8433,-300.00,13.45,0.2,-3228.00,16309,<NA>,16201,<NA>,<NA>
RF2-CC/2024/00034,28937,2024-02-21,posted,PdV SJC/3754 REEMBOLSO,PdV,6549,out_refund,<NA>,28811,90,...,10255,-4.00,61.26,0.1,-220.54,16310,<NA>,16202,<NA>,<NA>
RF2-CC/2024/00034,28937,2024-02-21,posted,PdV SJC/3754 REEMBOLSO,PdV,6549,out_refund,<NA>,28811,90,...,8301,-100.00,11.10,0.1,-999.00,16311,<NA>,16203,<NA>,<NA>
RF2-CC/2024/00034,28937,2024-02-21,posted,PdV SJC/3754 REEMBOLSO,PdV,6549,out_refund,<NA>,28811,90,...,10351,-3.00,64.50,0.1,-174.15,16312,<NA>,16204,<NA>,<NA>
RF2-CC/2024/00034,28937,2024-02-21,posted,PdV SJC/3754 REEMBOLSO,PdV,6549,out_refund,<NA>,28811,90,...,8511,-100.00,15.44,0.2,-1235.20,16313,<NA>,16205,<NA>,<NA>
RF2-CC/2024/00034,28937,2024-02-21,posted,PdV SJC/3754 REEMBOLSO,PdV,6549,out_refund,<NA>,28811,90,...,8352,-150.00,11.93,0.2,-1431.60,16314,<NA>,16206,<NA>,<NA>
RF2-CC/2024/00034,28937,2024-02-21,posted,PdV SJC/3754 REEMBOLSO,PdV,6549,out_refund,<NA>,28811,90,...,8341,-100.00,11.55,0.2,-924.00,16315,<NA>,16207,<NA>,<NA>


Se ocupa mejorar 'salesman_id'

In [34]:
check_salesman_id = len(complete_df.loc[complete_df['salesman_id'].isna()]) == 0
check_salesman_id_just_salesteams = len(complete_df.loc[~complete_df['salesman_id'].isin(list(descrip_sales_users_df['id']))]) == 0

# <span style="color:steelblue">Check Points<span>

In [36]:

check_1 =  len(fact_doc_df[fact_doc_df['module_origin'].isna()]) == 0
print('No hay valores sin calificar en module_origin:', check_1)
print('Las agrupaciones son del mismo tamaño:', check_total_size)
print('No hay diferencias al comparar cantidad de líneas por cada par de documentos:', check_each_document_size)
print('En el campo salesman_id sólo hay vendedoras', check_salesman_id)
print('Las vendedoras són únicamente ids que están dentro de los equipos de ventas', check_salesman_id_just_salesteams)

No hay valores sin calificar en module_origin: True
Las agrupaciones son del mismo tamaño: True
No hay diferencias al comparar cantidad de líneas por cada par de documentos: False
En el campo salesman_id sólo hay vendedoras False
Las vendedoras són únicamente ids que están dentro de los equipos de ventas False


In [35]:
check_all = check_1 and check_total_size and check_each_document_size and check_salesman_id and check_salesman_id_just_salesteams
print('Todos los check points están correctos:', check_all)

Todos los check points están correctos: False
